In [1]:
from lmfdb import db

INFO:LMFDB@2025-10-04 09:59:13,924: Configuration = {'flask_options': {'port': 37777, 'host': '127.0.0.1', 'debug': True, 'use_reloader': False}, 'postgresql_options': {'port': 5432, 'host': 'devmirror.lmfdb.xyz', 'dbname': 'lmfdb', 'user': 'lmfdb', 'password': '****'}, 'logging_options': {'logfile': 'flasklog', 'slowcutoff': 0.1, 'slowlogfile': 'slow_queries.log', 'editor': '', 'loglevel': 20}} 
INFO:LMFDB@2025-10-04 09:59:14,059: Connecting to PostgresSQL server as: user=lmfdb host=devmirror.lmfdb.xyz port=5432 dbname=lmfdb... 
INFO:LMFDB@2025-10-04 09:59:16,205: Done!
 connection = <connection object at 0x768ca934e200; dsn: 'user=lmfdb password=xxx dbname=lmfdb host=devmirror.lmfdb.xyz port=5432', closed: 0> 
2025-10-04 09:59:17,741 - SELECT pg_is_in_recovery() ran in  0.30677294731140137s 
2025-10-04 09:59:18,663 - SELECT current_setting('is_superuser') ran in  0.6147310733795166s 
INFO:LMFDB@2025-10-04 09:59:18,970: User: lmfdb 
INFO:LMFDB@2025-10-04 09:59:18,972: Read only: True 

# Download of all curves with growth of the torsion over some quadratic field

In [2]:
table = db.ec_torsion_growth.join_search(query={'degree' : int(2)}, 
                                        projection=["lmfdb_label", "torsion", "degree", "field", "conductor", ("ec_curvedata", "torsion_structure"),("ec_curvedata", "absD")], 
                                        join=[("ec_torsion_growth", "lmfdb_label", "ec_curvedata","lmfdb_label")],
                                         limit = 10)

# If you change the limit to None, it will take a few hours to download all the curves 
ecs = list(table)

2025-10-04 09:59:25,114 - SELECT count FROM "ec_torsion_growth_counts" WHERE cols = '["degree"]' AND values = '[2]' AND split = false ran in  0.5961771011352539s 
2025-10-04 09:59:26,343 - SELECT "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."torsion", "ec_torsion_growth"."degree", "ec_torsion_growth"."field", "ec_torsion_growth"."conductor", "ec_curvedata"."torsion_structure", "ec_curvedata"."absD" FROM "ec_torsion_growth" JOIN "ec_curvedata" ON "ec_torsion_growth"."lmfdb_label" = "ec_curvedata"."lmfdb_label" WHERE "ec_torsion_growth"."degree" = 2 ORDER BY "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."degree" LIMIT 1000 ran in  0.9218900203704834s 
2025-10-04 09:59:26,345 - Replicate with db.ec_torsion_growth.analyze({'degree': 2}, ['lmfdb_label', 'torsion', 'degree', 'field', 'conductor', ('ec_curvedata', 'torsion_structure'), ('ec_curvedata', 'absD')], 1000, 0)


In [3]:
#Prepare the data to state conjectures 
x = polygen(ZZ,'x')

for ec in ecs:
    ec['torsion_before'] = ec[('ec_curvedata','torsion_structure')]
    ec['torsion_after'] = ec['torsion']
    ec['curve_discriminant'] = ec[('ec_curvedata','absD')]
    
    del ec['torsion']
    del ec[('ec_curvedata','torsion_structure')]
    del ec[('ec_curvedata','absD')]
    
    f = ec['field']
    K.<alpha> = NumberField(f[0]*x^2+ f[1]*x + f[2]) 
    ec['disc_abs'] = K.absolute_discriminant()

In [4]:
ecs[0]

{'lmfdb_label': '100.a1',
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 100,
 'torsion_before': [2],
 'torsion_after': [2, 2],
 'curve_discriminant': 31250000,
 'disc_abs': 5}

# Funciones auxiliares

In [5]:
def prod(xs):
    prod = 1
    for x in xs:
        prod *= x
    return prod

def conditions(curve,  dict_key_property):
    boolean = True
    for key, propertyy in  dict_key_property.items():
        boolean = boolean and propertyy(curve[key])
    return boolean

def growth_torsion_search(dict_possible_torsion_growth, curves_list):
    return [ec for ec in curves_list if (tuple(ec['torsion_before']) in dict_possible_torsion_growth.keys()) 
            and (ec['torsion_after'] in dict_possible_torsion_growth[tuple(ec['torsion_before'])])]
    
    

In [6]:
dict_conditions = {'conductor' : lambda cond : cond%3 != 0, 
                   'torsion_before' : lambda tor_list : prod(tor_list)% 3 != 0,
                   'torsion_after' : lambda tor_list: prod(tor_list)%3 == 0,
                   'disc_abs' : lambda disc: disc%3 == 0}

ecs_trivial2C3 = [ec for ec in ecs if  conditions(ec, dict_conditions)]

In [7]:
counterexamples = [ec for ec in ecs_trivial2C3 if prod(ec['torsion_after'])/prod(ec['torsion_before']) != 3]
len(counterexamples)

0